In [1]:
# basic imports
import pandas as pd
import sys
import numpy as np
import pandas as pd
import scanpy as sc

# add `Tangram` to path
import tangram as tg

In [2]:
## setting for STARmap
sc_file_path = "/home/comp/cszrwang/data/osmfish/sc_osm.h5ad"
spatial_file_path = "/home/comp/cszrwang/data/osmfish/st_osm.h5ad"
celltype_key = 'celltype'
output_file_path = "/home/comp/cszrwang/results/Deconvolution/results/osmFISH/Tangram/"

In [3]:
ad_sc = sc.read_h5ad(sc_file_path)
ad_sp = sc.read_h5ad(spatial_file_path)

# use raw count both of scrna and spatial
sc.pp.normalize_total(ad_sc)
celltype_counts = ad_sc.obs[celltype_key].value_counts()
celltype_drop = celltype_counts.index[celltype_counts < 2]
print(f'Drop celltype {list(celltype_drop)} contain less 2 sample')
ad_sc = ad_sc[~ad_sc.obs[celltype_key].isin(celltype_drop),].copy()
sc.tl.rank_genes_groups(ad_sc, groupby=celltype_key, use_raw=False)
markers_df = pd.DataFrame(ad_sc.uns["rank_genes_groups"]["names"]).iloc[0:200, :]
print(markers_df)
genes_sc = np.unique(markers_df.melt().value.values)
print(genes_sc)
genes_st = ad_sp.var_names.values
genes = list(set(genes_sc).intersection(set(genes_st)))

Drop celltype [] contain less 2 sample
   L2/3 IT CTX-1 L4/5 IT CTX L5 IT CTX L5 NP CTX L5 PT CTX L6 CT CTX  \
0          Lamp5        Rorb     Aldoc     Aldoc     Lamp5      Syt6   
1           Anln      Kcnip2      Cnr1      Hexb     Aldoc      Tbr1   
2          Sox10       Lamp5      Ctps      Ctps      Cnr1      Hexb   
3          Cpne5         Ttr      Rorb      Rorb  Serpinf1    Kcnip2   
4            Ttr    Serpinf1      Tbr1     Itpr2      Anln       Vtn   
5          Foxj1      Pdgfra      Hexb      Mrc1     Acta2      Ctps   
6           Gfap       Foxj1     Tmem2     Tmem2     Tmem2     Itpr2   
7       Serpinf1        Plp1      Plp1      Gfap      Apln      Apln   
8           Apln        Gfap  Serpinf1      Flt1       Ttr      Flt1   
9          Acta2       Tmem2     Itpr2  Serpinf1    Pdgfra      Plp1   
10        Pdgfra       Mfge8      Mrc1     Acta2     Mfge8     Aldoc   
11           Vtn       Sox10    Pdgfra       Vtn      Gfap     Foxj1   
12         Mfge8        M

In [4]:
tg.pp_adatas(ad_sc, ad_sp, genes=genes)

ad_map = tg.map_cells_to_space(
                   ad_sc,
                   ad_sp,
                   mode='clusters',
                   cluster_label=celltype_key,)


tg.project_cell_annotations(ad_map, ad_sp, annotation=celltype_key)

celltype_density = ad_sp.obsm['tangram_ct_pred']
celltype_density = (celltype_density.T/celltype_density.sum(axis=1)).T

celltype_density.to_csv(output_file_path + '/Tangram_result.csv')

INFO:root:33 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:33 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.
INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 33 genes and rna_count_based density_prior in clusters mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.513, KL reg: 0.268
Score: 0.824, KL reg: 0.002
Score: 0.826, KL reg: 0.002
Score: 0.826, KL reg: 0.002
Score: 0.826, KL reg: 0.002
Score: 0.826, KL reg: 0.002
Score: 0.827, KL reg: 0.002
Score: 0.827, KL reg: 0.002
Score: 0.827, KL reg: 0.002
Score: 0.827, KL reg: 0.002


INFO:root:Saving results..
INFO:root:spatial prediction dataframe is saved in `obsm` `tangram_ct_pred` of the spatial AnnData.
